In [1]:
import tensorflow as tf 
import numpy as np 
from numpy import random
from keras.models import Sequential 
from keras.utils import np_utils 
from keras.layers import Dense,Activation,LSTM,Dropout,AveragePooling3D 
import pandas as pd 
import matplotlib.pyplot as plt 
from tensorflow import keras
import os
import glob

In [ ]:
#! pip install pandas 

In [3]:
from google.colab import drive 
drive.mount('/content/drive') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

# ! ls "/content/drive/"

In [ ]:
# ! ls "/content/drive/MyDrive/Colab Notebooks"

In [5]:

dataset_path='/content/drive/MyDrive/Colab Notebooks/dataset/' #path wher all the .txt files are located
files=os.listdir(dataset_path)

data_list=[]
for i in range(len(files)):
    tmp= np.loadtxt(dataset_path+files[i], delimiter=',', unpack=True)
    data_list.append(tmp)
data=np.asarray(data_list)    

df = pd.DataFrame(data)
df['filename']=files
df['y']=1 ## mmake all 1's then make normal subjects data to be 0's
df['y'][200:300]=0 ## O set--normal subjects
df['y'][400:500]=0  ## Z set--normal subjects


df.head()
print(df)

x=df.values
x_signals=x[:,1:(x.shape[1]-2)]
x_signals = np.asarray(x_signals).astype(np.float32)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


         0      1      2      3      4  ...   4094   4095   4096  filename  y
0     12.0   22.0   35.0   45.0   69.0  ...  -11.0    8.0   77.0  Z001.txt  1
1    -56.0  -50.0  -64.0  -91.0 -135.0  ... -173.0 -162.0  -82.0  Z002.txt  1
2    -37.0  -22.0  -17.0  -24.0  -31.0  ...    3.0    4.0   82.0  Z003.txt  1
3    -31.0  -43.0  -39.0  -39.0   -9.0  ...   -6.0   10.0   33.0  Z004.txt  1
4     14.0   26.0   32.0   25.0   16.0  ...  -32.0   -6.0  -17.0  Z005.txt  1
..     ...    ...    ...    ...    ...  ...    ...    ...    ...       ... ..
495  -40.0  -58.0  -75.0  -88.0  -89.0  ...  -13.0  -16.0 -151.0  S096.txt  0
496  187.0   44.0 -147.0 -368.0 -550.0  ...  703.0  446.0 -537.0  S097.txt  0
497 -438.0 -561.0 -622.0 -581.0 -460.0  ... -118.0 -163.0  -56.0  S098.txt  0
498 -476.0 -518.0 -521.0 -362.0  -68.0  ...  246.0  556.0  276.0  S099.txt  0
499   23.0  144.0  228.0  260.0  255.0  ... -155.0    6.0 -221.0  S100.txt  0

[500 rows x 4099 columns]


In [6]:
n_sample_points=400 #for every 2.3s segment the data
n_seg=x_signals.shape[1]/n_sample_points
data_seg_list=[]
data_seg_label_list=[]
for sig in range(x_signals.shape[0]):
  individual_signal=x_signals[sig,:]
  label=df['y'][sig]
  for seg in range(0,int(n_seg)):
    data_segmented=individual_signal[(seg*n_sample_points):((seg+1)*n_sample_points)]
    data_seg_list.append(data_segmented)
    data_seg_label_list.append(label)



In [7]:
data_seg=np.asarray(data_seg_list)
data_seg_label_=np.asarray(data_seg_label_list)
data_seg_label=np.empty((data_seg_label_.shape[0],1))
data_seg_label[:,0]=data_seg_label_


indices=np.arange(0,data_seg.shape[0])

shuffle_indices=random.permutation(indices)

In [8]:
###spilitting train and test by ratio 80:20
train_ratio=0.8
split_indices=data_seg.shape[0]*0.8

In [9]:
train_data=data_seg[shuffle_indices[0:int(split_indices)],:]
train_labels=data_seg_label[shuffle_indices[0:int(split_indices)],:]
test_data=data_seg[shuffle_indices[(int(split_indices)):-1],:]
test_labels=data_seg_label[shuffle_indices[(int(split_indices)):-1],:]


In [12]:
# ## adaptation of https://www.sciencedirect.com/science/article/pii/S0957417418302513#bib0050
# #Normalise the data using z-score normalisation
# train_data_normalised=(train_data-train_data.mean())/train_data.std()
# #conv-1
# inputs = tf.keras.Input(shape=(None,4000,400))
# conv1 = tf.keras.layers.Conv1D(24, (5),strides=1, padding='same',input_shape=(None,4000,400))(inputs)
# bn1=tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True,beta_initializer='zeros', gamma_initializer='ones')(conv1)
# rel1 = tf.keras.layers.Activation('relu')(bn1)

# #conv-2
# conv2 = tf.keras.layers.Conv1D(16, (3),strides=1, padding='same',input_shape=(None,24))(rel1)
# bn2=tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True,beta_initializer='zeros', gamma_initializer='ones')(conv2)
# rel2 = tf.keras.layers.Activation('relu')(bn2)

# #conv-3
# conv3 = tf.keras.layers.Conv1D(8, (3),strides=1, padding='same',input_shape=(None,16))(rel2)
# bn3=tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True,beta_initializer='zeros', gamma_initializer='ones')(conv3)
# rel3 = tf.keras.layers.Activation('relu')(bn3)

# #FC-1
# fc1=tf.keras.layers.Dense(20, activation='relu', use_bias=True)(rel3)

# #dropout
# do=tf.keras.layers.Dropout(rate=0.2)(fc1)

# #FC-2
# outputs = tf.keras.layers.Dense(2, activation=tf.nn.softmax)(do)
# ##build the model
# models = tf.keras.Model(inputs=inputs, outputs=outputs)

# models.compile(optimizer="Adam", loss="mse", metrics=["mae", "acc"])
# models.fit(train_data_normalised,train_labels,100,100)


In [70]:
## adaptation of https://www.sciencedirect.com/science/article/pii/S0957417418302513#bib0050
#Normalise the data using z-score normalisation
train_data_normalised=(train_data-train_data.mean())/train_data.std()
train_data_normalised = train_data_normalised.reshape((-1, 1, 400))
# train_data_normalised = train_data_normalised.reshape((1, 400, 4000))
# train_data_normalised = train_data_normalised.reshape((-1, 400, 1))
#conv-1
inputs = tf.keras.Input(shape=(None,400))
conv1 = tf.keras.layers.Conv1D(24, (5),strides=1, padding='same')(inputs)
bn1=tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True,beta_initializer='zeros', gamma_initializer='ones')(conv1)
rel1 = tf.keras.layers.Activation('relu')(bn1)

#conv-2
conv2 = tf.keras.layers.Conv1D(16, (3),strides=1, padding='same')(rel1)
bn2=tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True,beta_initializer='zeros', gamma_initializer='ones')(conv2)
rel2 = tf.keras.layers.Activation('relu')(bn2)

#conv-3
conv3 = tf.keras.layers.Conv1D(8, (3),strides=1, padding='same')(rel2)
bn3=tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True,beta_initializer='zeros', gamma_initializer='ones')(conv3)
rel3 = tf.keras.layers.Activation('relu')(bn3)

#FC-1
fc1=tf.keras.layers.Dense(20, activation='relu', use_bias=True)(rel3)

#dropout
do=tf.keras.layers.Dropout(rate=0.1)(fc1)

#FC-2
outputs = tf.keras.layers.Dense(2, activation=tf.nn.softmax)(do)
##build the model
models = tf.keras.Model(inputs=inputs, outputs=outputs)


In [81]:
models.compile(optimizer="sgd", loss="mse", metrics=[ "acc"])
models.fit(train_data_normalised,train_labels,1,epochs=100)

Epoch 1/100
4000/4000 [==============================] - 6s 1ms/step - loss: 0.2500 - acc: 0.5869
Epoch 2/100
4000/4000 [==============================] - 6s 1ms/step - loss: 0.2500 - acc: 0.5911
Epoch 3/100
4000/4000 [==============================] - 6s 1ms/step - loss: 0.2500 - acc: 0.5919
Epoch 4/100
4000/4000 [==============================] - 6s 1ms/step - loss: 0.2500 - acc: 0.6002
Epoch 5/100
4000/4000 [==============================] - 6s 1ms/step - loss: 0.2500 - acc: 0.5945
Epoch 6/100
4000/4000 [==============================] - 6s 1ms/step - loss: 0.2500 - acc: 0.5914
Epoch 7/100
4000/4000 [==============================] - 6s 1ms/step - loss: 0.2500 - acc: 0.5870
Epoch 8/100
4000/4000 [==============================] - 6s 1ms/step - loss: 0.2500 - acc: 0.5963
Epoch 9/100
4000/4000 [==============================] - 6s 1ms/step - loss: 0.2500 - acc: 0.5947
Epoch 10/100
4000/4000 [==============================] - 6s 1ms/step - loss: 0.2500 - acc: 0.5994
Epoch 11/100
4000/4

In [82]:
test_data_1 = test_data.reshape((-1, 1, 400))
models.evaluate(test_data_1, test_labels,1)

999/999 [==============================] - 1s 950us/step - loss: 0.4942 - acc: 0.4014


[0.4942488372325897, 0.4014014005661011]

In [86]:
classify=models.predict(test_data_1)
classify

array([[[6.6366893e-01, 3.3633104e-01]],

       [[1.0000000e+00, 1.3224137e-12]],

       [[1.0000000e+00, 7.6489749e-17]],

       ...,

       [[1.0000000e+00, 5.3110923e-27]],

       [[0.0000000e+00, 1.0000000e+00]],

       [[1.0000000e+00, 0.0000000e+00]]], dtype=float32)

In [ ]:
# model = Sequential()
# model.add(tf.keras.layers.Conv1D( 24, 5,strides=1, padding='same',input_shape=(None,4000,400)))
# model.add(tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True,beta_initializer='zeros', gamma_initializer='ones'))
# model.add(Activation('relu'))

# model.add(tf.keras.layers.Conv1D(16, 3,strides=1, padding='same',input_shape=(None,24)))
# model.add(tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True,beta_initializer='zeros', gamma_initializer='ones'))
# model.add(tf.keras.layers.Activation('relu'))

# model.add(tf.keras.layers.Conv1D(8,3,strides=1, padding='same',input_shape=(None,16)))
# model.add(tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True,beta_initializer='zeros', gamma_initializer='ones'))
# model.add(tf.keras.layers.Activation('relu'))

# model.add(tf.keras.layers.Dense(20, activation='relu', use_bias=True))

# model.add(tf.keras.layers.Dropout(rate=0.2))

# model.add(tf.keras.layers.Dense(2, activation=tf.nn.softmax))

# model.compile(optimizer="Adam", loss="mse", metrics=["mae", "acc"])
# model.fit(train_data_normalised,train_labels,100,100)


In [47]:
#dd=tf.data.Dataset.from_tensor_slices((train_data_normalised,train_labels))


In [ ]:
# x = tf.keras.layers.Dense(4, activation=tf.nn.relu)(inputs)
# outputs = tf.keras.layers.Dense(5, activation=tf.nn.softmax)(x)
# model = tf.keras.Model(inputs=inputs, outputs=outputs)

# input_shape = (4, 10, 128)
# x = tf.random.normal(input_shape)
# y = tf.keras.layers.Conv1D(
# 32, 3, activation='relu',input_shape=input_shape[1:])(x)


# inputs = tf.keras.Input(shape=(4000,400))
# conv1 = tf.keras.layers.Conv1D(24, 5,strides=1, padding='same',input_shape=(4000,400))(inputs)
# bn1=tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True,beta_initializer='zeros', gamma_initializer='ones')(conv1)
# rel1 = tf.keras.layers.Activation('relu')(bn1)




In [ ]:
# ##old model using the relu
# model=Sequential()
# model.add(Conv2D(32, (3,3), activation='relu'))
# model.add(Conv2D(32, (3,3), activation='relu'))
# model.add(BatchNormalization())

# model.add(BatchNormalization(input_shape=(2,)))

# model.add(Dense(256,input_shape=(400,)))
# model.add(Activation('relu'))
# model.add(Dense(128))
# model.add(Activation('relu'))
# model.add(Dense(128))
# model.add(Activation('relu'))
# model.add(Dense(64))
# model.add(Activation('relu'))
# model.add(Dense(32))
# model.add(Activation('relu'))
# model.add(Dense(32))
# model.add(Activation('relu'))
# model.add(Dense(32))
# model.add(Activation('relu'))
# model.add(Dense(16))
# model.add(Activation('relu'))
# model.add(Dense(16))
# model.add(Activation('relu'))
# model.add(Dense(8))
# model.add(Activation('relu'))
# model.add(Dense(2))
# model.add(Activation('softmax'))
# model.summary()

# model.compile(optimizer="Adam", loss="mse", metrics=["mae", "acc"])
# model.fit(train_data,train_labels,10,10)

In [ ]:
# opt = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
# var = tf.Variable(1.0)
# val0 = var.value()
# loss = lambda: (var ** 2)/2.0         # d(loss)/d(var1) = var1
# # First step is `- learning_rate * grad`
# step_count = opt.minimize(loss, [var]).numpy()
# val1 = var.value()
# (val0 - val1).numpy()

# # On later steps, step-size increases because of momentum
# step_count = opt.minimize(loss, [var]).numpy()
# val2 = var.value()
# (val1 - val2).numpy()

